# Préparation de données pour visualisations
Le fichiers de données après préparation a une taille d'environ 160 Mo; c'est trop important pour être déposé sur github.com. Nous décidons toutefois de déposer quelques fichiers synthétiques pour les visualisations de donnnées. Ces données sont utilisées par les feuilles streamlit.

In [2]:
import pandas as pd
import json


In [3]:
rep_src   = "../data/raw/"       # Fichiers téléchargés avant traitements
rep_inter = "../data/inter/"     # Des fichiers intermédiaires sont utilisés
rep_dst   = "../data/processed/" # Fichiers utilsés par la modélisation
rep_ref   = "../references/"

# Lecture de la description des fichiers avec nom, séparateurs
with open (rep_ref + "desc_fic_raw.json", "rt", encoding='utf-8') as fichier:
    desc_fic_raw = json.load(fichier)

with open ("../references/desc_vars.json", "rt", encoding='utf-8') as fichier:
    desc_vars = json.load(fichier)

dfrub = {}

In [4]:

desc_fic_raw["usagers"]["fichiers"]

[{'nom': 'usagers_2005.csv', 'sep': ',', 'phase': '1'},
 {'nom': 'usagers_2006.csv', 'sep': ',', 'phase': '1'},
 {'nom': 'usagers_2007.csv', 'sep': ',', 'phase': '1'},
 {'nom': 'usagers_2008.csv', 'sep': ',', 'phase': '1'},
 {'nom': 'usagers_2009.csv', 'sep': ',', 'phase': '1'},
 {'nom': 'usagers_2010.csv', 'sep': ',', 'phase': '1'},
 {'nom': 'usagers_2011.csv', 'sep': ',', 'phase': '1'},
 {'nom': 'usagers_2012.csv', 'sep': ',', 'phase': '1'},
 {'nom': 'usagers_2013.csv', 'sep': ',', 'phase': '1'},
 {'nom': 'usagers_2014.csv', 'sep': ',', 'phase': '1'},
 {'nom': 'usagers_2015.csv', 'sep': ',', 'phase': '1'},
 {'nom': 'usagers_2016.csv', 'sep': ',', 'phase': '1'},
 {'nom': 'usagers-2017.csv', 'sep': ',', 'phase': '1'},
 {'nom': 'usagers-2018.csv', 'sep': ',', 'phase': '1'},
 {'nom': 'usagers-2019.csv', 'sep': ';', 'phase': '2'},
 {'nom': 'usagers-2020.csv', 'sep': ';', 'phase': '2'},
 {'nom': 'usagers-2021.csv', 'sep': ';', 'phase': '2'},
 {'nom': 'usagers-2022.csv', 'sep': ';', 'phase'

In [5]:
dfl = []
nb_obs = 0
for fichier_usagers in desc_fic_raw["usagers"]["fichiers"] :
    dtype = fichier_usagers.get("dtype")
    # lecture du fichier
    df = pd.read_csv(rep_src + fichier_usagers["nom"],
                     sep=fichier_usagers["sep"],
                     dtype=dtype,
                     encoding="latin_1",
                     index_col=False,
                     quotechar="\"",
                     low_memory=False)

    nb_obs += df.shape[0]
    if fichier_usagers.get("rename_cols") is not None:
        df = df.rename(columns=fichier_usagers.get("rename_cols"))
        print("  - rename ", fichier_usagers.get("rename_cols"))
    print(fichier_usagers["nom"], df.shape)  # Pour info et mise au point
    dfl.append(df)

dfu =  pd.concat(dfl)  
print("Nombre de DataFrames  : ", len(dfl))
print("Nombre d'observations : ", nb_obs, dfu.shape[0])
print("Colonnes du DataFrame : ", dfu.shape[1])


usagers_2005.csv (197498, 12)
usagers_2006.csv (187085, 12)
usagers_2007.csv (188457, 12)
usagers_2008.csv (170960, 12)
usagers_2009.csv (165962, 12)
usagers_2010.csv (154192, 12)
usagers_2011.csv (148543, 12)
usagers_2012.csv (138628, 12)
usagers_2013.csv (128694, 12)
usagers_2014.csv (132186, 12)
usagers_2015.csv (130378, 12)
usagers_2016.csv (133422, 12)
usagers-2017.csv (136021, 12)
usagers-2018.csv (130169, 12)
usagers-2019.csv (132977, 15)
usagers-2020.csv (105295, 15)
usagers-2021.csv (129248, 16)
usagers-2022.csv (126662, 16)
usagers-2023.csv (125789, 16)
Nombre de DataFrames  :  19
Nombre d'observations :  2762166 2762166
Colonnes du DataFrame :  17


# Historique gravité sur 19 ans
Le code de cette cellule charge les données des usagers sur 19 ans. Ces données servent uniquement à l'affichage d'un historique.

In [7]:

dfe = dfu[["Num_Acc", "grav"]]  # dfe contient 
dfe = dfe.dropna()
dfe["grav"] = dfe["grav"].astype(str)
dfe = dfe[dfe["grav"] != -1]   # Suppression des observations pour lesquelles la gravité est -1 càd. inconnue
dfe["annee"] = dfe["Num_Acc"].astype(str).str[:4]
dfe = dfe.drop("Num_Acc", axis = 1)

dfea = dfe.pivot_table(index='annee', columns='grav', aggfunc='size', fill_value=0)
dfea = dfea.sort_values(by='annee').reset_index(drop=False)
dfea = dfea.rename(columns = {"1" : "grav_1", "2" : "grav_2", "3" : "grav_3", "4" : "grav_4", })

dfea = dfea.drop ("-1", axis = 1)
#print (dfea.columns)
#display(dfea)

dfea.to_csv("../data/processed/evol_grav.csv", sep = '\t', index=False)


In [8]:
display(dfea)  # Pour contrôle 

grav,annee,grav_1,grav_2,grav_3,grav_4
0,2005,80272,5543,40792,70891
1,2006,76163,4942,41869,64111
2,2007,76910,4838,39754,66955
3,2008,69612,4443,36179,60726
4,2009,67806,4443,34506,59207
5,2010,62847,4172,31337,55836
6,2011,60560,4111,30635,53237
7,2012,56577,3842,28107,50102
8,2013,52622,3427,26895,45750
9,2014,53487,3557,27502,47640


In [9]:
# Chargement des 4 rubriques de 2019 à 2022 inclus.
# Le séparateur est la tabulation.
# Le paramètre low_memory = False est utilisé car avant préprocessing
# les types des données ne sont pas harmonisés.

dfc = pd.read_csv(rep_inter + "caracteristiques_raw_4.csv", sep = '\t', low_memory = False)
dfl = pd.read_csv(rep_inter + "lieux_raw_4.csv",            sep = '\t', low_memory = False)
dfu = pd.read_csv(rep_inter + "usagers_raw_4.csv",          sep = '\t', low_memory = False)
dfv = pd.read_csv(rep_inter + "vehicules_raw_4.csv",        sep = '\t', low_memory = False)

# Affichages des dimensions des DataFrames des rubriques pour contrôle
print (f"Taille caracteristiques : {dfc.shape[0]} x {dfc.shape[1]}")
print (f"Taille lieux            : {dfl.shape[0]} x {dfl.shape[1]}")
print (f"Taille usagers          : {dfu.shape[0]} x {dfu.shape[1]}")
print (f"Taille vehicules        : {dfv.shape[0]} x {dfv.shape[1]}")

Taille caracteristiques : 218404 x 16
Taille lieux            : 218404 x 19
Taille usagers          : 494182 x 17
Taille vehicules        : 373584 x 12


In [10]:

# Création d'un DataFrame synthétique vide servant à contenir des stats destinées à l'affichage
df_synth = pd.DataFrame(columns=["rubrique", "variable", "modalite", "count"])

# Spécification des types de données pour chaque colonne
df_synth = df_synth.astype({"rubrique": "str", "variable" : "str", "modalite" : "str", "count" : "int"})

df_synth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   rubrique  0 non-null      object
 1   variable  0 non-null      object
 2   modalite  0 non-null      object
 3   count     0 non-null      int64 
dtypes: int64(1), object(3)
memory usage: 132.0+ bytes


In [11]:

dict_visu = {"caractéristiques" : {"df" : dfc, "variables" : ["lum" , "atm"]},
             "lieux" :            {"df" : dfl, "variables" : ["catr" , "circ", "surf", "situ"]},
             "usagers" :          {"df" : dfu, "variables" : ["trajet", "locp", "grav", "secu1", "secu2", "secu3"]},
             "véhicules" :        {"df" : dfv, "variables" : ["catv", "choc"]}}

for rubrique, desc in dict_visu.items():
    df_rub = desc.get("df")
    print (f"Rubrique \"{rubrique}\", dimension : {df_rub.shape[0]} x {df_rub.shape[1]}")
    
    for variable in desc.get("variables"):
        #print ()
        print (f"    Variable : {variable}")
        df_rub_var = pd.DataFrame(df_rub[variable].value_counts())
        df_rub_var["modalite"] = df_rub_var.index
        df_rub_var["modalite"] = df_rub_var["modalite"].astype(str)
        rempl_mod = desc_vars.get(variable).get("values")
        df_rub_var["modalite"] = df_rub_var["modalite"].replace (rempl_mod)
        df_rub_var["rubrique"] = rubrique
        df_rub_var["variable"] = variable
        #print (df_rub_var)
        df_synth = pd.concat([df_rub_var, df_synth])

display (df_synth.head(20))

df_synth.to_csv(rep_dst + "stats_expl_var.csv", sep = '\t')


Rubrique "caractéristiques", dimension : 218404 x 16
    Variable : lum
    Variable : atm
Rubrique "lieux", dimension : 218404 x 19
    Variable : catr
    Variable : circ
    Variable : surf
    Variable : situ
Rubrique "usagers", dimension : 494182 x 17
    Variable : trajet
    Variable : locp
    Variable : grav
    Variable : secu1
    Variable : secu2
    Variable : secu3
Rubrique "véhicules", dimension : 373584 x 12
    Variable : catv
    Variable : choc


,count,modalite,rubrique,variable
1,134490,Avant,véhicules,choc
3,55117,Avant gauche,véhicules,choc
2,45406,Avant droit,véhicules,choc
4,37060,Arrière,véhicules,choc
8,25984,Côté gauche,véhicules,choc
0,23912,Aucun,véhicules,choc
7,22266,Côté droit,véhicules,choc
6,13240,Arrière gauche,véhicules,choc
5,10631,Arrière droit,véhicules,choc
9,5281,Chocs multiples (tonneaux),véhicules,choc
